In [2]:
!pip install evaluate datasets transformers torch transformers[torch] SentencePiece rouge_score bleu bert-score squad sacrebleu sacremoses tqdm pypdfium2 spacy unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.5/424.5 kB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 90.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [3]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments, get_linear_schedule_with_warmup,pipeline
import torch
import pandas as pd
import numpy as np
from datasets import load_dataset,Dataset
import os
from evaluate import load
import evaluate
from tqdm import tqdm
import sys
from pathlib import Path
from torch.utils.data import DataLoader, TensorDataset
import pypdfium2 as pdfium
import re
from spacy.lang.en import English
from unstructured.cleaners.core import group_broken_paragraphs, replace_unicode_quotes, clean, clean_non_ascii_chars, remove_punctuation
from sklearn.model_selection import train_test_split

In [4]:
dataset = load_dataset("asset","ratings")
print(dataset)
dataset1 = load_dataset("asset")
print(dataset1)
train_x = dataset['full']['original']
train_y = dataset['full']['simplification']
validation_x = dataset1['validation']['original']
validation_y = dataset1['validation']['simplifications']
test_x = dataset1['test']['original']
test_y = dataset1['test']['simplifications']

train_df = pd.DataFrame({
    'source_text': train_x,
    'target_text': train_y
})
eval_df = pd.DataFrame({
    'source_text': validation_x,
    'target_text': validation_y
})
test_df = pd.DataFrame({
    'source_text': test_x,
    'target_text': test_y
})
# Add the prefix to source_text
train_df['source_text'] = "simplify this sentence: " + train_df['source_text']

eval_df['source_text'] = "simplify this sentence: " + eval_df['source_text']

test_df['source_text'] = "simplify this sentence: " + test_df['source_text']


Extracting data files:   0%|          | 0/23 [00:00<?, ?it/s]

Generating full split:   0%|          | 0/4500 [00:00<?, ? examples/s]

DatasetDict({
    full: Dataset({
        features: ['original', 'simplification', 'original_sentence_id', 'aspect', 'worker_id', 'rating'],
        num_rows: 4500
    })
})


Extracting data files:   0%|          | 0/23 [00:00<?, ?it/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/359 [00:00<?, ? examples/s]

DatasetDict({
    validation: Dataset({
        features: ['original', 'simplifications'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['original', 'simplifications'],
        num_rows: 359
    })
})


In [ ]:


print("Train DataFrame:")
for index, row in train_df.head().iterrows():
    print(f"Index: {index}")
    print(f"Source Text: {row['source_text']}")
    print(f"Target Text: {row['target_text']}")
    print("\n")

print("Eval DataFrame:")
for index, row in eval_df.head().iterrows():
    print(f"Index: {index}")
    print(f"Source Text: {row['source_text']}")
    print(f"Target Text: {row['target_text']}")
    print("\n")

print("Test DataFrame:")
for index, row in test_df.head().iterrows():
    print(f"Index: {index}")
    print(f"Source Text: {row['source_text']}")
    print(f"Target Text: {row['target_text']}")
    print("\n")


Train DataFrame:
Index: 0
Source Text: simplify this sentence: Since 2000, the recipient of the Kate Greenaway medal has also been presented with the Colin Mears award to the value of £5000.
Target Text: Since 2000, the winner of the Kate Greenaway medal has also been given to the Colin Mears award of the Kate Greenaway medal.


Index: 1
Source Text: simplify this sentence: Since 2000, the recipient of the Kate Greenaway medal has also been presented with the Colin Mears award to the value of £5000.
Target Text: Since 2000, the winner of the Kate Greenaway medal has also been given to the Colin Mears award of the Kate Greenaway medal.


Index: 2
Source Text: simplify this sentence: Since 2000, the recipient of the Kate Greenaway medal has also been presented with the Colin Mears award to the value of £5000.
Target Text: Since 2000, the winner of the Kate Greenaway medal has also been given to the Colin Mears award of the Kate Greenaway medal.


Index: 3
Source Text: simplify this sente

In [ ]:
model_name = "t5-small"  # Change to the desired T5 model size
tokenizer = T5Tokenizer.from_pretrained(model_name)

train_encodings = tokenizer(train_df['source_text'].tolist() , max_length=512, truncation=True, padding=True, return_tensors="pt" )
train_labels = tokenizer(train_df['target_text'].tolist() , max_length=512, truncation=True, padding=True, return_tensors="pt")

validation_encodings = tokenizer(eval_df['source_text'].tolist() , max_length=512, truncation=True, padding=True, return_tensors="pt")
validation_labels = tokenizer(eval_df['target_text'].tolist() , max_length=512, truncation=True, padding=True, return_tensors="pt" )

test_encodings = tokenizer(test_df['source_text'].tolist() , max_length=512, truncation=True, padding=True, return_tensors="pt")
test_labels = tokenizer(test_df['target_text'].tolist() , max_length=512, truncation=True, padding=True, return_tensors="pt")

train_dataset = Dataset.from_dict({"input_ids": train_encodings["input_ids"], "labels": train_labels["input_ids"]})
validation_dataset = Dataset.from_dict({"input_ids": validation_encodings["input_ids"], "labels": validation_labels["input_ids"]})
test_dataset = Dataset.from_dict({"input_ids": test_encodings["input_ids"], "labels": test_labels["input_ids"]})

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
model_name = "t5-small"
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/T5_small",
    overwrite_output_dir=True,
    per_device_train_batch_size=4,
    # per_device_eval_batch_size=1,
    num_train_epochs=4,
    learning_rate=1e-4,
    push_to_hub=False,
    save_strategy="epoch",
)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    optimizers = (optimizer,None)
)

trainer.train()

***** Running training *****
  Num examples = 4500
  Num Epochs = 4
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 4500


Step,Training Loss
500,0.288900
1000,0.035200
1500,0.012400
2000,0.006900
2500,0.005500
3000,0.004000
3500,0.002900
4000,0.002900
4500,0.002300


Saving model checkpoint to /content/drive/MyDrive/T5_small/checkpoint-1125
Configuration saved in /content/drive/MyDrive/T5_small/checkpoint-1125/config.json
Model weights saved in /content/drive/MyDrive/T5_small/checkpoint-1125/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/T5_small/checkpoint-2250
Configuration saved in /content/drive/MyDrive/T5_small/checkpoint-2250/config.json
Model weights saved in /content/drive/MyDrive/T5_small/checkpoint-2250/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/T5_small/checkpoint-3375
Configuration saved in /content/drive/MyDrive/T5_small/checkpoint-3375/config.json
Model weights saved in /content/drive/MyDrive/T5_small/checkpoint-3375/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/T5_small/checkpoint-4500
Configuration saved in /content/drive/MyDrive/T5_small/checkpoint-4500/config.json
Model weights saved in /content/drive/MyDrive/T5_small/checkpoint-4500/pytorch_model.bin


Training c

TrainOutput(global_step=4500, training_loss=0.04011359151204427, metrics={'train_runtime': 411.8477, 'train_samples_per_second': 43.705, 'train_steps_per_second': 10.926, 'total_flos': 323551494144000.0, 'train_loss': 0.04011359151204427, 'epoch': 4.0})

In [ ]:
model_name = "t5-base"  # Change to the desired T5 model size
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

train_encodings = tokenizer(train_df['source_text'].tolist() , max_length=512, truncation=True, padding=True, return_tensors="pt" )
train_labels = tokenizer(train_df['target_text'].tolist() , max_length=512, truncation=True, padding=True, return_tensors="pt")

validation_encodings = tokenizer(eval_df['source_text'].tolist() , max_length=512, truncation=True, padding=True, return_tensors="pt")
validation_labels = tokenizer(eval_df['target_text'].tolist() , max_length=512, truncation=True, padding=True, return_tensors="pt" )

test_encodings = tokenizer(test_df['source_text'].tolist() , max_length=512, truncation=True, padding=True, return_tensors="pt")
test_labels = tokenizer(test_df['target_text'].tolist() , max_length=512, truncation=True, padding=True, return_tensors="pt")

train_dataset = Dataset.from_dict({"input_ids": train_encodings["input_ids"], "labels": train_labels["input_ids"]})
validation_dataset = Dataset.from_dict({"input_ids": validation_encodings["input_ids"], "labels": validation_labels["input_ids"]})
test_dataset = Dataset.from_dict({"input_ids": test_encodings["input_ids"], "labels": test_labels["input_ids"]})

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/T5_base",
    overwrite_output_dir=True,
    per_device_train_batch_size=4,
    # per_device_eval_batch_size=1,
    num_train_epochs=4,
    learning_rate=1e-4,
    push_to_hub=False,
    save_strategy="epoch",
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    optimizers = (optimizer,None)
)

# Train the model
trainer.train()

***** Running training *****
  Num examples = 4500
  Num Epochs = 4
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 4500


Step,Training Loss
500,0.150200
1000,0.005800
1500,0.003300
2000,0.001600
2500,0.001500
3000,0.001000
3500,0.000500
4000,0.000700
4500,0.000500


Saving model checkpoint to /content/drive/MyDrive/T5_base/checkpoint-1125
Configuration saved in /content/drive/MyDrive/T5_base/checkpoint-1125/config.json
Model weights saved in /content/drive/MyDrive/T5_base/checkpoint-1125/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/T5_base/checkpoint-2250
Configuration saved in /content/drive/MyDrive/T5_base/checkpoint-2250/config.json
Model weights saved in /content/drive/MyDrive/T5_base/checkpoint-2250/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/T5_base/checkpoint-3375
Configuration saved in /content/drive/MyDrive/T5_base/checkpoint-3375/config.json
Model weights saved in /content/drive/MyDrive/T5_base/checkpoint-3375/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/T5_base/checkpoint-4500
Configuration saved in /content/drive/MyDrive/T5_base/checkpoint-4500/config.json
Model weights saved in /content/drive/MyDrive/T5_base/checkpoint-4500/pytorch_model.bin


Training completed. Do

TrainOutput(global_step=4500, training_loss=0.01834034866756863, metrics={'train_runtime': 1241.66, 'train_samples_per_second': 14.497, 'train_steps_per_second': 3.624, 'total_flos': 1455789957120000.0, 'train_loss': 0.01834034866756863, 'epoch': 4.0})

In [ ]:
# prompt: write code for evaluate each checkpoint /content/drive/MyDrive/T5_base/ of with validation_dataset  and then test it using test_dataset

rouge = evaluate.load('rouge')
bert_score = evaluate.combine(["bertscore"])
google_bleu = evaluate.combine(["google_bleu"])
sacrebleu = evaluate.combine(["sacrebleu"])
sari = evaluate.combine(["sari"])
bleu = evaluate.combine(["bleu"])
def compute_score(pred,rec,sor):
  bert_score_all= bert_score.compute(predictions=pred, references=rec,lang="en")
  P = bert_score_all['precision']
  R = bert_score_all['recall']
  F1 = bert_score_all['f1']
  P = sum(P)/len(P)
  F1 = sum(F1)/len(F1)
  R = sum(R)/len(R)
  bertscore_score = {"precision": P, "recall": R, "f1": F1}
  google_bleu_score = google_bleu.compute(predictions=pred, references=rec)
  sacrebleu_score = sacrebleu.compute(predictions=pred, references=rec)
  sari_score = sari.compute(predictions=pred, references=rec,sources=sor)
  rouge_score = rouge.compute(predictions=pred, references=rec)
  bleu_score = bleu.compute(predictions=pred, references=rec)
  return {"rouge":rouge_score, "bleu":bleu_score,
    "google_bleu":google_bleu_score,
    "bertscore":bertscore_score,
    "sacrebleu":sacrebleu_score,
    "sari":sari_score,
          }
dataset = load_dataset("asset","ratings")
print(dataset)
dataset1 = load_dataset("asset")
print(dataset1)
train_x = dataset['full']['original']
train_y = dataset['full']['simplification']
validation_x = dataset1['validation']['original']
validation_y = dataset1['validation']['simplifications']
test_x = dataset1['test']['original']
test_y = dataset1['test']['simplifications']

train_df = pd.DataFrame({
    'source_text': train_x,
    'target_text': train_y
})
eval_df = pd.DataFrame({
    'source_text': validation_x,
    'target_text': validation_y
})
test_df = pd.DataFrame({
    'source_text': test_x,
    'target_text': test_y
})
# Add the prefix to source_text
train_df['source_text'] = "simplify this sentence: " + train_df['source_text']

eval_df['source_text'] = "simplify this sentence: " + eval_df['source_text']

test_df['source_text'] = "simplify this sentence: " + test_df['source_text']


validation_dataset = Dataset.from_dict({"text":eval_df['source_text'],"label":eval_df['target_text']})
test_dataset = Dataset.from_dict({"text":test_df['source_text'],"label":test_df['target_text']})

model_name = "T5-base"

tokenizer = T5Tokenizer.from_pretrained(model_name,skip_special_tokens=True)

dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)

generation_parameters = {"max_length":512}
# Evaluate each checkpoint
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
checkpoints_dir = "/content/drive/MyDrive/NLP_MODEL/"
for checkpoint in sorted(os.listdir(checkpoints_dir)):
    if checkpoint.startswith("checkpoint"):
        model_path = os.path.join(checkpoints_dir, checkpoint)

        pipe = pipeline(model = model_path,tokenizer=tokenizer,task = "text2text-generation",device=device,max_new_tokens= 100)

        predict = []
        for text in tqdm(dataloader, desc=f"Evaluating {checkpoint}"):
          predictions = pipe(text["text"])

          predict.append(predictions[0]["generated_text"])

        result = compute_score(predict,test_dataset["label"],test_dataset["text"])

        print(f"Results for {checkpoint}:")
        print("result:",result)


DatasetDict({
    full: Dataset({
        features: ['original', 'simplification', 'original_sentence_id', 'aspect', 'worker_id', 'rating'],
        num_rows: 4500
    })
})
DatasetDict({
    validation: Dataset({
        features: ['original', 'simplifications'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['original', 'simplifications'],
        num_rows: 359
    })
})


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Evaluating checkpoint-3375:   3%|▎         | 10/359 [00:05<04:16,  1.36it/s]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Evaluating checkpoint-3375: 100%|██████████| 359/359 [02:57<00:00,  2.02it/s]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Results for checkpoint-3375:
result: {'rouge': {'rouge1': 0.8694396010589851, 'rouge2': 0.7680403106356468, 'rougeL': 0.8562113629503249, 'rougeLsum': 0.8556885665257417}, 'bleu': {'bleu': 0.8616609505841286, 'precisions': [0.9516384778012685, 0.8911083368012207, 0.8350364963503649, 0.7784624865197967], 'brevity_penalty': 1.0, 'length_ratio': 1.7997621878715815, 'translation_length': 7568, 'reference_length': 4205}, 'google_bleu': {'google_bleu': 0.7025755654900914}, 'bertscore': {'precision': 0.9753523223247369, 'recall': 0.9760546055012759, 'f1': 0.9742366877438963}, 'sacrebleu': {'score': 86.16609505841288, 'counts': [7202, 6424, 5720, 5053], 'totals': [7568, 7209, 6850, 6491], 'precisions': [95.16384778012684, 89.11083368012207, 83.5036496350365, 77.84624865197966], 'bp': 1.0, 'sys_len': 7568, 'ref_len': 7459}, 'sari': {'sari': 50.337260880472336}}


In [5]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load the trained model and tokenizer
model_path = "/content/drive/MyDrive/NLP_MODEL/checkpoint-3375/"  # Update this path to the directory where your model is saved
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained("t5-base")

input_sentence = "simplify this sentence: Adjacent counties are Marin (to the south), Mendocino (to the north), Lake (northeast), Napa (to the east), and Solano and Contra Costa (to the southeast)."

# Tokenize the input sentence
input_ids = tokenizer(input_sentence, return_tensors="pt").input_ids

# Generate the simplified output
output_ids = model.generate(input_ids,max_new_tokens = 512)

# Decode the output into text
output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Print the results
print("Input Sentence:", input_sentence)
print("Simplified Output:", output_text)

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:240: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the

Input Sentence: simplify this sentence: Adjacent counties are Marin (to the south), Mendocino (to the north), Lake (northeast), Napa (to the east), and Solano and Contra Costa (to the southeast).
Simplified Output: Adjacent counties are Marin (to the south), Mendocino (to the north), Lake (northeast), Napa (to the east), and Solano and Contra Costa (to the southeast).


In [6]:
def remove_unnecessary_tokens(token):
    token = re.sub(r"[^0-9a-z]", "", token)
    return token.isdigit()

print("intit complete")

model_path = "/content/drive/MyDrive/NLP_MODEL/checkpoint-3375/"

tokenizer = T5Tokenizer.from_pretrained("t5-base",skip_special_tokens=True)

print("model loaded")
nlp = English()
nlp.add_pipe("sentencizer")

file_list = ["Can Large Language Models Explain Themselves.pdf"]
print("file list")
print(file_list)
allowed_punctuation = {'!', '@', '#', '$', '%', '^', '&', '*',
                       '(', ')', '-', '=', '_', '+', '`', '~',
                       '[', ']', '{', '}', '\\', '|', "'", '"',';', ':', ',', '.', '<', '>', '?', '/'}
allowed_entities = {'DATE',
'EVENT',
'FAC',
'GPE',
'LANGUAGE',
'LAW',
'LOC',
'MONEY',
'NORP',
'ORDINAL',
'ORG',
'PERCENT',
'PERSON',
'PRODUCT',
'QUANTITY',
'TIME',
'WORK_OF_ART'}

device = "cuda" if torch.cuda.is_available() else "cpu"

pipe = pipeline(model = model_path,tokenizer=tokenizer,task = "text2text-generation",device=device,max_new_tokens= 100)

for i, fname in enumerate(file_list):
    print(i+1)
    file_stem = fname
    print(fname)
    pdf = pdfium.PdfDocument(f"{file_stem}")
    whole_text = ""
    n_pages = len(pdf)
    for num in range(n_pages):
        page = pdf[num]
        textpage = page.get_textpage()
        text_all = textpage.get_text_range()
        textpage = text_all
        textpage = clean_non_ascii_chars(textpage)
        textpage = replace_unicode_quotes(textpage)
        textpage = textpage.encode("utf-8")
        textpage = textpage.decode("utf-8")
        whole_text += '\n' + textpage + '\n'
    count = 0
    ls = []
    doc = nlp(whole_text)
    for sen in doc.sents:
        sen = str(sen)
        sen = sen.strip()
        if not remove_unnecessary_tokens(sen):
            filtered_sentence = ""
            filtered_sentence = sen
            filtered_sentence = group_broken_paragraphs(filtered_sentence)
            filtered_sentence = filtered_sentence.strip()
            doc1=nlp(filtered_sentence)
            filtered_sentenc = []

            for token in doc1:
                if (
                    (token.text.isalnum() and token.text.isdigit()) or
                    (token.text.isalnum() and token.text.isalpha()) or
                    (token.text in allowed_punctuation) or
                    ((token.ent_type_ != "") and (token.ent_type_ !=  "CARDINAL"))
                ):
                    filtered_sentenc.append(token.text + token.whitespace_)
            filtered_sentence = "".join(filtered_sentenc)

            filtered_sentence = clean(filtered_sentence,extra_whitespace=True)
            if(filtered_sentence.strip()):
                ls.append(filtered_sentence.strip())
                count += 1

    print(f"Number of sentences : {count}")
    print("\n")
    outputs = []
    for text in tqdm(ls, desc=f"Evaluating {fname}"):
      text = "simplify this sentence: "+ text

      output = pipe(text)
      outputs.append(output[0]["generated_text"])
for out in outputs:
  print(out)

intit complete


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


model loaded
file list
['Can Large Language Models Explain Themselves.pdf']
1
Can Large Language Models Explain Themselves.pdf
Number of sentences : 542




Evaluating Can Large Language Models Explain Themselves.pdf: 100%|██████████| 542/542 [12:00<00:00,  1.33s/it]

Large Language Models Explain Themselves?
A Study of LLM-Generated Self-Explanations Shiyuan Huang Siddarth Mamidanna Shreedhar Jangam Yilun Zhou Leilani Gilpin UC Santa Cruz UC Santa Cruz MIT CSAIL UC Santa Cruz , spmamida, , AbstractLarge language models (LLMs) such as ChatGPT have demonstrated superior performance on a variety of natural language processing
Also, since these models are instruction-tuned on human conversations to produce helpful responses, they can and often will produce explanations along with the response, which we call selfexplanations.
For example, when analyzing the sentiment of a movie review, the model may output not only the positivity of the sentiment, but also an explanation (, by listing the sentimentladen words such as fantastic and memorable in the review).
How good are these automatically generated self-explanations?
In this paper, we investigate this question on the task of sentiment analysis and for feature attribution explanation, one of the most com

In [ ]:
dataset = load_dataset("asset","ratings")
print(dataset)
dataset1 = load_dataset("asset")
print(dataset1)
train_x = dataset['full']['original']
train_y = dataset['full']['simplification']
validation_x = dataset1['validation']['original']
validation_y = dataset1['validation']['simplifications']
test_x = dataset1['test']['original']
test_y = dataset1['test']['simplifications']

train_df = pd.DataFrame({
    'source_text': train_x,
    'target_text': train_y
})
eval_df = pd.DataFrame({
    'source_text': validation_x,
    'target_text': validation_y
})
test_df = pd.DataFrame({
    'source_text': test_x,
    'target_text': test_y
})
# Add the prefix to source_text
train_df['source_text'] = "simplify this sentence: " + train_df['source_text']

eval_df['source_text'] = "simplify this sentence: " + eval_df['source_text']

test_df['source_text'] = "simplify this sentence: " + test_df['source_text']


DatasetDict({
    full: Dataset({
        features: ['original', 'simplification', 'original_sentence_id', 'aspect', 'worker_id', 'rating'],
        num_rows: 4500
    })
})
DatasetDict({
    validation: Dataset({
        features: ['original', 'simplifications'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['original', 'simplifications'],
        num_rows: 359
    })
})


In [ ]:
!pip install accelerate transformers[torch]

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration

training_args = Seq2SeqTrainingArguments(
    output_dir="/content/BART_large",
    overwrite_output_dir=True,
    per_device_train_batch_size=4,
    # per_device_eval_batch_size=1,
    num_train_epochs=4,
    learning_rate=1e-4,
    push_to_hub=False,
    save_strategy="epoch",
)


In [ ]:
model_name = "facebook/bart-large"    # Change to the desired T5 model size
tokenizer = BartTokenizer.from_pretrained(model_name)

train_encodings = tokenizer(train_df['source_text'].tolist() , max_length=512, truncation=True, padding=True, return_tensors="pt" )
train_labels = tokenizer(train_df['target_text'].tolist() , max_length=512, truncation=True, padding=True, return_tensors="pt")

# validation_encodings = tokenizer(eval_df['source_text'].tolist() , max_length=512, truncation=True, padding=True, return_tensors="pt")
# validation_labels = tokenizer(eval_df['target_text'].tolist() , max_length=512, truncation=True, padding=True, return_tensors="pt" )

# test_encodings = tokenizer(test_df['source_text'].tolist() , max_length=512, truncation=True, padding=True, return_tensors="pt")
# test_labels = tokenizer(test_df['target_text'].tolist() , max_length=512, truncation=True, padding=True, return_tensors="pt")

train_dataset = Dataset.from_dict({"input_ids": train_encodings["input_ids"], "labels": train_labels["input_ids"]})
# validation_dataset = Dataset.from_dict({"input_ids": validation_encodings["input_ids"], "labels": validation_labels["input_ids"]})
# test_dataset = Dataset.from_dict({"input_ids": test_encodings["input_ids"], "labels": test_labels["input_ids"]})

In [ ]:

# tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    optimizers = (optimizer,None)
)

trainer.train()

Step,Training Loss
500,0.634500
1000,0.070700
1500,0.037300
2000,0.022300
2500,0.013100
3000,0.008900


TrainOutput(global_step=3375, training_loss=0.11721397774307817, metrics={'train_runtime': 1438.5002, 'train_samples_per_second': 9.385, 'train_steps_per_second': 2.346, 'total_flos': 1828494508032000.0, 'train_loss': 0.11721397774307817, 'epoch': 3.0})

In [ ]:
# prompt: write code for evaluate each checkpoint /content/drive/MyDrive/T5_base/ of with validation_dataset  and then test it using test_dataset

rouge = evaluate.load('rouge')
bert_score = evaluate.combine(["bertscore"])
google_bleu = evaluate.combine(["google_bleu"])
sacrebleu = evaluate.combine(["sacrebleu"])
sari = evaluate.combine(["sari"])
bleu = evaluate.combine(["bleu"])
def compute_score(pred,rec,sor):
  bert_score_all= bert_score.compute(predictions=pred, references=rec,lang="en")
  P = bert_score_all['precision']
  R = bert_score_all['recall']
  F1 = bert_score_all['f1']
  P = sum(P)/len(P)
  F1 = sum(F1)/len(F1)
  R = sum(R)/len(R)
  bertscore_score = {"precision": P, "recall": R, "f1": F1}
  google_bleu_score = google_bleu.compute(predictions=pred, references=rec)
  sacrebleu_score = sacrebleu.compute(predictions=pred, references=rec)
  sari_score = sari.compute(predictions=pred, references=rec,sources=sor)
  rouge_score = rouge.compute(predictions=pred, references=rec)
  bleu_score = bleu.compute(predictions=pred, references=rec)
  return {"rouge":rouge_score, "bleu":bleu_score,
    "google_bleu":google_bleu_score,
    "bertscore":bertscore_score,
    "sacrebleu":sacrebleu_score,
    "sari":sari_score,
          }
dataset = load_dataset("asset","ratings")
print(dataset)
dataset1 = load_dataset("asset")
print(dataset1)
train_x = dataset['full']['original']
train_y = dataset['full']['simplification']
validation_x = dataset1['validation']['original']
validation_y = dataset1['validation']['simplifications']
test_x = dataset1['test']['original']
test_y = dataset1['test']['simplifications']

train_df = pd.DataFrame({
    'source_text': train_x,
    'target_text': train_y
})
eval_df = pd.DataFrame({
    'source_text': validation_x,
    'target_text': validation_y
})
test_df = pd.DataFrame({
    'source_text': test_x,
    'target_text': test_y
})
# Add the prefix to source_text
train_df['source_text'] = "simplify this sentence: " + train_df['source_text']

eval_df['source_text'] = "simplify this sentence: " + eval_df['source_text']

test_df['source_text'] = "simplify this sentence: " + test_df['source_text']


validation_dataset = Dataset.from_dict({"text":eval_df['source_text'],"label":eval_df['target_text']})
test_dataset = Dataset.from_dict({"text":test_df['source_text'],"label":test_df['target_text']})

model_name = "facebook/bart-large"

tokenizer = BartTokenizer.from_pretrained(model_name,skip_special_tokens=True)

dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)

generation_parameters = {"max_length":512}
# Evaluate each checkpoint
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
checkpoints_dir = "/content/BART_large"
for checkpoint in sorted(os.listdir(checkpoints_dir)):
    if checkpoint.startswith("checkpoint"):
        model_path = os.path.join(checkpoints_dir, checkpoint)

        pipe = pipeline(model = model_path,tokenizer=tokenizer,task = "text2text-generation",device=device,max_new_tokens= 100)

        predict = []
        for text in tqdm(dataloader, desc=f"Evaluating {checkpoint}"):
          predictions = pipe(text["text"])

          predict.append(predictions[0]["generated_text"])

        result = compute_score(predict,test_dataset["label"],test_dataset["text"])

        print(f"Results for {checkpoint}:")
        print("result:",result)


DatasetDict({
    full: Dataset({
        features: ['original', 'simplification', 'original_sentence_id', 'aspect', 'worker_id', 'rating'],
        num_rows: 4500
    })
})
DatasetDict({
    validation: Dataset({
        features: ['original', 'simplifications'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['original', 'simplifications'],
        num_rows: 359
    })
})


Evaluating checkpoint-1125:   3%|▎         | 10/359 [00:04<03:46,  1.54it/s]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Evaluating checkpoint-1125: 100%|██████████| 359/359 [02:35<00:00,  2.31it/s]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Results for checkpoint-1125:
result: {'rouge': {'rouge1': 0.7922299356503901, 'rouge2': 0.6703534507224055, 'rougeL': 0.7781079488932218, 'rougeLsum': 0.7777658775703689}, 'bleu': {'bleu': 0.7722337307677061, 'precisions': [0.9108544350658966, 0.8152955896152643, 0.732062966031483, 0.6541578576462297], 'brevity_penalty': 1.0, 'length_ratio': 1.6059453032104638, 'translation_length': 6753, 'reference_length': 4205}, 'google_bleu': {'google_bleu': 0.5867575047902917}, 'bertscore': {'precision': 0.9608218996969771, 'recall': 0.9574678311108878, 'f1': 0.9566968005347717}, 'sacrebleu': {'score': 77.05203218180485, 'counts': [6151, 5213, 4418, 3713], 'totals': [6753, 6394, 6035, 5676], 'precisions': [91.08544350658967, 81.52955896152643, 73.2062966031483, 65.41578576462297], 'bp': 0.9977812301102745, 'sys_len': 6753, 'ref_len': 6768}, 'sari': {'sari': 45.67404523435875}}


Evaluating checkpoint-2250: 100%|██████████| 359/359 [02:19<00:00,  2.57it/s]


Results for checkpoint-2250:
result: {'rouge': {'rouge1': 0.7365004459295301, 'rouge2': 0.6231708494376939, 'rougeL': 0.7204214045783768, 'rougeLsum': 0.7211901952345706}, 'bleu': {'bleu': 0.8037237640168741, 'precisions': [0.9385789569325822, 0.843895800933126, 0.7650992685475444, 0.688572719060524], 'brevity_penalty': 1.0, 'length_ratio': 1.308680142687277, 'translation_length': 5503, 'reference_length': 4205}, 'google_bleu': {'google_bleu': 0.523891940702837}, 'bertscore': {'precision': 0.9489667664663374, 'recall': 0.931397543478145, 'f1': 0.9378140520584617}, 'sacrebleu': {'score': 76.0113588689271, 'counts': [5165, 4341, 3661, 3049], 'totals': [5503, 5144, 4785, 4428], 'precisions': [93.85789569325823, 84.3895800933126, 76.50992685475444, 68.85727190605239], 'bp': 0.9457398458524517, 'sys_len': 5503, 'ref_len': 5810}, 'sari': {'sari': 42.20598896481773}}


Evaluating checkpoint-3375: 100%|██████████| 359/359 [02:31<00:00,  2.36it/s]


Results for checkpoint-3375:
result: {'rouge': {'rouge1': 0.7581173008269494, 'rouge2': 0.6383196201477845, 'rougeL': 0.7408301266687882, 'rougeLsum': 0.7411293646862518}, 'bleu': {'bleu': 0.7701782119325984, 'precisions': [0.9063250600480385, 0.8075093442065919, 0.7296906097340329, 0.6588622291021672], 'brevity_penalty': 1.0, 'length_ratio': 1.4851367419738406, 'translation_length': 6245, 'reference_length': 4205}, 'google_bleu': {'google_bleu': 0.5525511318145425}, 'bertscore': {'precision': 0.9530121172703077, 'recall': 0.943289543260771, 'f1': 0.9456390002975888}, 'sacrebleu': {'score': 75.03359588996798, 'counts': [5660, 4753, 4033, 3405], 'totals': [6245, 5886, 5527, 5168], 'precisions': [90.63250600480384, 80.75093442065919, 72.96906097340329, 65.88622291021672], 'bp': 0.9742368029561251, 'sys_len': 6245, 'ref_len': 6408}, 'sari': {'sari': 43.783801606876985}}
